# Testing Calibration V2Worksheet

## 1. Setup
This section load python objects and variables please check the ```cam``` variable

### 1.1 Load python objects
Loads ```scope``` and ```lamp``` python objects for reading sekonic and communication with the lamp

In [1]:
cam = "http://192.168.2.224/video_feed"
sharedFolder = r"C:\Users\Virtual Production 2\Documents\ApolloLogs\\"
%run calibration_setup_v2.ipynb

pygame 2.1.2 (SDL 2.0.18, Python 3.10.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


### 1.2 Connect to lamp
Lamp needs to be closed before it can be opened again.
If you dont know the serial port where the lamp is connected use the Find COM port section
1st You need to setup sekonic and feedback camera (the areas where the data will be read, 1st -green value is set up so it would be possibel to get dUV - sign location), then close the lamp and start again with the scope objects

In [48]:
lamp1.close()

In [35]:
lamp1 = apollo_device(ComPort='COM13')
lamp1.getId()
lamp1.macId = lamp1.macId.replace(":","")
lamp1.macId

'C45BBE2D4448'

In [36]:
lamp1.color = [200,50,50,500,0]
lamp1.update()

In [5]:
cmd = "I 1 1 1 255 0\n"
lamp1.send(cmd.encode('utf-8'))
lamp1.read()
print(lamp1.lastValAsJsonString)

{"mixed_red":-1,"mixed_green":-1,"mixed_blue":-1,"mixed_white":-1,"red_actual":137,"lamp_temp":39.750000,"lamp_target":65.000000,"fan_speed":0,"Timestamp":1700981467657258200}


#### [Optional] Find COM port

In [9]:
# Get COM port list
import serial.tools.list_ports

com_ports = list(serial.tools.list_ports.comports())

for port, desc, hwid in com_ports:
    print(str(port)+" "+str(desc))

COM5 USB Serial Port (COM5)
COM6 USB Serial Port (COM6)


### 1.3 Set Sekonic camera areas
Set the area where to "read" the Tcp, dUV and lux variables
Once the object is initalized tune the paramters and test by ```update()```
To save the paramteres use the Setup below to generate the ```saved_string``` and copy and paste it into funtion paramter and save the workbook

In [31]:
Brightness = scope(saved_string="self.slider_h.value = (232, 391)\nself.slider_v.value = (319, 377)\nself.minus_comp.value = False\nself.minus_brightness.value = '150'\nself.offset_x.value = 0\nself.offset_y.value = 0\nself.rota.value = '90'\nself.flip.value = False\nself.flop.value = False\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.')]) \n")

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Label(value='Out:')

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x87\x00\x00\x002\x08\x02\x00\x00\x00\xf9\x82_\x9…

Output()

In [32]:
Brightness.update()

10800

In [33]:
dUV = scope(saved_string="self.slider_h.value = (258, 447)\nself.slider_v.value = (237, 295)\nself.minus_comp.value = True\nself.minus_brightness.value = '150'\nself.offset_x.value = -85\nself.offset_y.value = 3\nself.rota.value = '90'\nself.flip.value = False\nself.flop.value = False\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.'), ('20.0', '0,0'), ('0.6', '0.0'), ('0.8', '0.0')]) \n")

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Label(value='Out:')

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x87\x00\x00\x002\x08\x02\x00\x00\x00\xf9\x82_\x9…

Output()

In [38]:
dUV.update()

-0.0352

In [39]:
Kelvin = scope(saved_string="self.slider_h.value = (259, 417)\nself.slider_v.value = (159, 217)\nself.minus_comp.value = False\nself.minus_brightness.value = '150'\nself.offset_x.value = 0\nself.offset_y.value = 0\nself.rota.value = '90'\nself.flip.value = False\nself.flop.value = False\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.'), ('G', '6')]) \n")

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Label(value='Out:')

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x87\x00\x00\x002\x08\x02\x00\x00\x00\xf9\x82_\x9…

Output()

In [40]:
Kelvin.update()

5181

#### Setup

In [49]:
Brightness.save_string()

"self.slider_h.value = (232, 391)\nself.slider_v.value = (319, 377)\nself.minus_comp.value = False\nself.minus_brightness.value = '150'\nself.offset_x.value = 0\nself.offset_y.value = 0\nself.rota.value = '90'\nself.flip.value = False\nself.flop.value = False\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.')]) \n"

In [50]:
dUV.save_string()

"self.slider_h.value = (258, 447)\nself.slider_v.value = (237, 295)\nself.minus_comp.value = True\nself.minus_brightness.value = '150'\nself.offset_x.value = -85\nself.offset_y.value = 3\nself.rota.value = '90'\nself.flip.value = False\nself.flop.value = False\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.'), ('20.0', '0,0'), ('0.6', '0.0'), ('0.8', '0.0')]) \n"

In [51]:
Kelvin.save_string()

"self.slider_h.value = (259, 417)\nself.slider_v.value = (159, 217)\nself.minus_comp.value = False\nself.minus_brightness.value = '150'\nself.offset_x.value = 0\nself.offset_y.value = 0\nself.rota.value = '90'\nself.flip.value = False\nself.flop.value = False\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.'), ('G', '6')]) \n"

# Test

In [44]:
lamp1.close()
time.sleep(0.5)

In [45]:
lamp1 = apollo_device(ComPort='COM13',SekBrightnes=Brightness, SekDuv=dUV, SekKelvin=Kelvin)
lamp1.getId()
lamp1.macId = lamp1.macId.replace(":","")
lamp1.macId

'C45BBE2D4448'

In [ ]:
%%time

startCalib = 0
endCalib = 5
readCalibration = True
writeCalibration = True
allowedKelvinDifference = 20
allowedDuvDifference = 0.0002
maxTries = 10

data = None

def calRange(KelvinTemp = 2, BrightnessV = 8, readCalibration = True, writeCalibration = True):
    global calibration_points
    if readCalibration:
        print("Loading calibartion from lamp\n")
        lamp1.send(("Cr"+str(KelvinTemp)+" "+str(BrightnessV)+" \n").encode('utf-8'))
        lamp1.read()
        jsonString = "{"+lamp1.lastValAsJsonString[1:-1]+"}"
        jsonObject = eval(jsonString)

        print("loaded values: "+str(jsonObject)+"\n")
        calibration_points[KelvinTemp][8][2] = jsonObject['r']
        calibration_points[KelvinTemp][8][3] = jsonObject['g']
        calibration_points[KelvinTemp][8][4] = jsonObject['b']
        calibration_points[KelvinTemp][8][5] = jsonObject['w']

        # The answer is 2 line, so to clear the buffer
        lamp1.read()

    calibration_points = lamp1.AdjustLux(BrightnessV,KelvinTemp, calibration_points)
    calibration_points = lamp1.AdjustDuv(BrightnessV,KelvinTemp, calibration_points)
    calibration_points = lamp1.AdjustKelvin(BrightnessV,KelvinTemp, calibration_points)

    #write calibration to lamp
    if writeCalibration:
        print("Writing calibratrion:\n")
        cmd = "Cw"+str(KelvinTemp)+" "+str(BrightnessV)+" "+str(calibration_points[KelvinTemp][BrightnessV][2])+" "+str(calibration_points[KelvinTemp][BrightnessV][3])+" "+str(calibration_points[KelvinTemp][BrightnessV][4])+" "+str(calibration_points[KelvinTemp][BrightnessV][5])+"\n"
        lamp1.send(cmd.encode('utf-8'))
        print(cmd)
        lamp1.read()
        print(lamp1.lastValAsJsonString)

    print(f'Temperature: {lamp1.SekKelvin.update()}K\ndUv: {lamp1.SekDuv.update()}\nBrightness: {lamp1.SekBrightnes.update()}lx')
    return [lamp1.SekKelvin.lastVal, lamp1.SekDuv.lastVal, lamp1.SekBrightnes.lastVal]

# try N times to get the dUV & Kelvin difference low enough
for kelvin in range(startCalib,endCalib+1):
    print(f"starting kelvin range {KelvinRangeIds[kelvin]}\n")
    done = False
    for i in range(maxTries):
        if done:
            break  # Exit the inner loop when calibration is successful
        measured = calRange(KelvinTemp=kelvin, readCalibration=readCalibration, writeCalibration=writeCalibration)
        if (abs(measured[1]) < allowedDuvDifference) and (abs(measured[0]-KelvinRangeIds[kelvin]) < allowedKelvinDifference):
            done = True
    if done:
        # Record final results for proofing
        new_data_line = {
            'wb': [KelvinRangeIds[kelvin]],
            'brightness_point': [8],
            'kelvin': [measured[0]],
            'duv': [measured[1]],
            'lux': [measured[2]]
        }
        new_data_line_df = pd.DataFrame(new_data_line)
        print("Debug new_data_line: " + str(new_data_line))
        if data is None:
            data = new_data_line_df
        else:
            data = pd.concat([data, new_data_line_df], ignore_index=True)
        

starting kelvin range 2800

Loading calibartion from lamp

loaded values: {'wb': 0, 'brightness': 8, 'r': 1181, 'g': 668, 'b': 9, 'w': 449, 'Timestamp': 1700937282620243000}

Adjusting lamp brightness to 10000lx in 2800K with [3, 1, 0, 1]/[26, 15, 0, 10] steps
current brightness: 10500, ideal 10000, diff -500 : [1181, 668, 9, 449, 0] / [757, 668, 9, 449, 0]
current brightness: 10300, ideal 10000, diff -300 : [1155, 653, 9, 439, 0] / [741, 653, 9, 439, 0]
current brightness: 10000, ideal 10000, diff 0 : [1129, 638, 9, 429, 0] / [724, 638, 9, 429, 0]
selected: [2800, 8, 1129, 638, 9, 429]
Adjusting lamp dUV to +/- 0.003 in 2800K with [0, 1, 0, 0]/[0, 30, 0, 0] steps
current dUV: -0.0007, ideal 0.0000, diff -0.0007 : [1129, 638, 9, 429, 0] / [726, 638, 9, 429, 0]
current dUV: -0.0007, ideal 0.0000, diff -0.0007 : [1129, 639, 9, 429, 0] / [726, 639, 9, 429, 0]
current dUV: -0.0007, ideal 0.0000, diff -0.0007 : [1129, 640, 9, 429, 0] / [728, 640, 9, 429, 0]
current dUV: -0.0008, ideal 0.000

In [42]:
data = None

def calRange(KelvinTemp = 2, BrightnessV = 8, readCalibration = True, writeCalibration = True):
    global calibration_points
    if readCalibration:
        print("Loading calibartion from lamp\n")
        lamp1.send(("Cr"+str(KelvinTemp)+" "+str(BrightnessV)+" \n").encode('utf-8'))
        lamp1.read()
        jsonString = "{"+lamp1.lastValAsJsonString[1:-1]+"}"
        jsonObject = eval(jsonString)

        print("loaded values: "+str(jsonObject)+"\n")
        calibration_points[KelvinTemp][8][2] = jsonObject['r']
        calibration_points[KelvinTemp][8][3] = jsonObject['g']
        calibration_points[KelvinTemp][8][4] = jsonObject['b']
        calibration_points[KelvinTemp][8][5] = jsonObject['w']

        # The answer is 2 line, so to clear the buffer
        lamp1.read()

    calibration_points = lamp1.AdjustLux(BrightnessV,KelvinTemp, calibration_points)
    calibration_points = lamp1.AdjustDuv(BrightnessV,KelvinTemp, calibration_points)
    calibration_points = lamp1.AdjustKelvin(BrightnessV,KelvinTemp, calibration_points)

    #write calibration to lamp
    if writeCalibration:
        print("Writing calibratrion:\n")
        cmd = "Cw"+str(KelvinTemp)+" "+str(BrightnessV)+" "+str(calibration_points[KelvinTemp][BrightnessV][2])+" "+str(calibration_points[KelvinTemp][BrightnessV][3])+" "+str(calibration_points[KelvinTemp][BrightnessV][4])+" "+str(calibration_points[KelvinTemp][BrightnessV][5])+"\n"
        lamp1.send(cmd.encode('utf-8'))
        print(cmd)
        lamp1.read()
        print(lamp1.lastValAsJsonString)

    print(f'Temperature: {lamp1.SekKelvin.update()}K\ndUv: {lamp1.SekDuv.update()}\nBrightness: {lamp1.SekBrightnes.update()}lx')
    return [lamp1.SekKelvin.lastVal, lamp1.SekDuv.lastVal, lamp1.SekBrightnes.lastVal]


In [47]:
%%time

startCalib = 5
endCalib = 5
readCalibration = True
writeCalibration = True
maxTries = 10
allowedRanges = [
    [0,0.0000,0],
    [1,0.0050,500],
    [2,0.0010,300],
    [3,0.0005,100],
    [4,0.0005,100],
    [5,0.0005,100],
    [6,0.0005,100],
    [7,0.0005,100]]

    


def calKelvinRange(KelvinTemp = 2, readCalibration = True, writeCalibration = True):
    global calibration_points, data

    if readCalibration:
        print("Loading calibartion from lamp\n")
        lamp1.send(("Cr"+str(KelvinTemp)+" 8 \n").encode('utf-8'))
        lamp1.read()
        jsonString = "{"+lamp1.lastValAsJsonString[1:-1]+"}"
        jsonObject = eval(jsonString)

        print("loaded values: "+str(jsonObject)+"\n")
        calibration_points[KelvinTemp][8][2] = jsonObject['r']
        calibration_points[KelvinTemp][8][3] = jsonObject['g']
        calibration_points[KelvinTemp][8][4] = jsonObject['b']
        calibration_points[KelvinTemp][8][5] = jsonObject['w']

        # The answer is 2 line, so to clear the buffer
        lamp1.read()

    for lx in reversed(range(1,8)):
        calibration_points[KelvinTemp][lx] = [round(x/2) for x in calibration_points[KelvinTemp][lx+1]]
        done = False
        
        # try N times to get the dUV & Kelvin difference low enough
        for i in range(maxTries):
            if done:
                break
            measured = calRange(KelvinTemp=KelvinTemp,BrightnessV=lx,readCalibration=False,writeCalibration=writeCalibration)
            if (abs(measured[1]) < allowedRanges[lx][1]) and (abs(measured[0]-KelvinRangeIds[kelvin]) < allowedRanges[lx][2]):
                done = True
        
        if done:
            # record final results for proofing
            newDataLine = {
                'wb': [KelvinRangeIds[KelvinTemp]],
                'brightness_point': [lx],
                'kelvin': [measured[0]],
                'duv': [measured[1]],
                'lux': [measured[2]]
            }
            newDataLineDf = pd.DataFrame(newDataLine)
            print("Debug newDataLine: "+str(newDataLine))
            if data is None:
                data = newDataLineDf
            else:
                data = pd.concat([data, newDataLineDf], ignore_index=True)
        
# 
for kelvin in range(startCalib,endCalib+1):
    print(f"starting kelvin range {KelvinRangeIds[kelvin]}\n")
    calKelvinRange(KelvinTemp=kelvin,readCalibration=readCalibration,writeCalibration=writeCalibration)
    

starting kelvin range 10000

Loading calibartion from lamp

loaded values: {'wb': 5, 'brightness': 8, 'r': 441, 'g': 781, 'b': 381, 'w': 1929, 'Timestamp': 1701012287123317700}

Adjusting lamp brightness to 10000lx in 10000K with [1, 2, 1, 4]/[10, 18, 9, 44] steps
current brightness: 2210, ideal 10000, diff 7790 : [220, 390, 190, 964, 0] / [155, 390, 190, 964, 0]
current brightness: 2010, ideal 10000, diff 7990 : [230, 408, 199, 1008, 0] / [162, 408, 199, 1008, 0]
current brightness: 1810, ideal 10000, diff 8190 : [240, 426, 208, 1052, 0] / [169, 426, 208, 1052, 0]
current brightness: 9900, ideal 10000, diff 100 : [250, 444, 217, 1096, 0] / [176, 444, 217, 1096, 0]
current brightness: 10400, ideal 10000, diff -400 : [251, 446, 218, 1100, 0] / [177, 446, 218, 1100, 0]
current brightness: 10800, ideal 10000, diff -800 : [241, 428, 209, 1056, 0] / [170, 428, 209, 1056, 0]
current brightness: 10900, ideal 10000, diff -900 : [231, 410, 200, 1012, 0] / [164, 410, 200, 1012, 0]
current bright

ValueError: [[31mERROR[0m] Could get value even after several tries, just giving up

In [ ]:
import datetime

# Save Log
file_path = sharedFolder+"Apollo"+str(lamp1.macId)+"_"+datetime.datetime.now().strftime("%d%m%Y_%H%M%S")+"caliblog.csv"
data.to_csv(file_path, index=False)
print("Saved to: "+file_path)

# Save Calibration
with open(sharedFolder+"Apollo"+str(lamp1.macId)+"_"+datetime.datetime.now().strftime("%d%m%Y_%H%M%S")+"calib.py", "w") as file:
    file.write(str(calibration_points))


In [ ]:
from mitosheet import *; register_analysis("id-uhsskvzpjn");

# Filtered duv
data = data[data['duv'] > 0.001]

# Formatted dataframes. View these styling objects to see the formatted dataframe
data_styler = data.style\
    .format("{:.4f}", subset=['duv'])


In [ ]:
playSucess()

In [ ]:
'''
cmd = "Cr5 8\n"
lamp1.send(cmd.encode('utf-8'))
lamp1.read()
print(lamp1.lastValAsJsonString)
'''

In [ ]:
'''
cmd = "Cw5 1 100 30 30 21 \n"
lamp1.send(cmd.encode('utf-8'))
lamp1.read()
print(lamp1.lastValAsJsonString)
'''